In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.autograd.function as F
import torch.utils.data as Data
torch.__version__

'1.1.0'

In [2]:
BATCH_SIZE = 64

In [3]:
def sample_data_1():
    count = 100000
    rand = np.random.RandomState(0)
    return [[1.0, 2.0]] + rand.randn(count, 2) * [[5.0, 1.0]]
def sample_data_2():
    count = 100000
    rand = np.random.RandomState(0)
    return [[1.0, 2.0]] + (rand.randn(count, 2) * [[5.0, 1.0]]).dot(
        [[np.sqrt(2) / 2, np.sqrt(2) / 2], [-np.sqrt(2) / 2, np.sqrt(2) / 2]])

In [4]:
torch.cuda.is_available()

False

In [5]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
data1 = np.asarray(sample_data_1(), dtype=np.float32)
data2 = np.asarray(sample_data_2(), dtype=np.float32)

In [7]:
np.min(data1)

-23.260588

In [8]:
data1_tensor = torch.from_numpy(data1).to(device)
data2_tensor = torch.from_numpy(data2).to(device)

In [9]:
dset1 = Data.TensorDataset(data1_tensor)
dset2 = Data.TensorDataset(data2_tensor)

In [10]:
dset1_loader = Data.DataLoader(
    dataset=dset1,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=1,
)

dset2_loader = Data.DataLoader(
    dataset=dset2,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=1,
)

In [12]:
for idx, [batch] in enumerate(dset1_loader):
    print(batch.size())
    break
    

torch.Size([64, 2])


In [15]:
class SimpleVAE(nn.Module):
    def __init__(self, var_dim):
        super(SimpleVAE, self).__init__()
        self.var_dim = var_dim
        self.fc1 = nn.Linear(2, 32)
        self.enc_mean = nn.Linear(32, 8)
        self.enc_logvar = nn.Linear(32, self.var_dim)
        self.dec_1 = nn.Linear(8, 32)
        self.dec_2 = nn.Linear(32, 2)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.enc_mean(h1), self.enc_logvar(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.dec_1(z))
        return self.dec_2(h3)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [16]:
vae = SimpleVAE(8)

In [12]:
vae(data1_tensor[:1])

NameError: name 'vae' is not defined

In [ ]:
data1_tensor[:1]

In [ ]:
data1_tensor